In [1]:
import numpy as np
import h5py,time, matplotlib.pyplot as plt
from scipy.optimize import fmin_cg, minimize
from drift.core import manager
from matplotlib.colors import LogNorm
import corrcal2
import sys
sys.path.insert(0,'/home/zahra/PIPELINE')
from log_red_cal_new import Visibilities_grid, Bls_counts, colour_scatterplot, Scatterplot
from decimal import Decimal
from operator import add,sub
from scipy.optimize import LinearConstraint, BFGS, Bounds
import scipy as sp


In [2]:

ts_2=h5py.File('/media/zahra/DATA/5by5_dishvar_pt1/draco_synthesis/maps/tstream_2.h5','r')
ag_2=h5py.File('/media/zahra/DATA/5by5_dishvar_pt1/draco_synthesis/rand_gains/amp_phase/app_gain_2.h5','r')
rg_2=h5py.File('/media/zahra/DATA/5by5_dishvar_pt1/draco_synthesis/rand_gains/sig_2.h5','r')

gn_2=h5py.File('/media/zahra/DATA/5by5_dishvar_pt1/draco_synthesis/rand_gains/ts_final/app_gain_noise_2.h5','r')

#ts_2=h5py.File('/media/zahra/DATA/example_5by5_lmax_200_no_scatter/draco_synthesis_no_fluc/maps/tstream_2.h5','r')
#ag_2=h5py.File('/media/zahra/DATA/example_5by5_lmax_200_no_scatter/draco_synthesis_no_fluc/rand_gains/amp_phase/app_gain_2.h5','r')
#rg_2=h5py.File('/media/zahra/DATA/example_5by5_lmax_200_no_scatter/draco_synthesis_no_fluc/rand_gains/sig_2.h5','r')

#gn_2=h5py.File('/media/zahra/DATA/example_5by5_lmax_200_no_scatter/draco_synthesis_no_fluc/rand_gains/ts_final/app_gain_noise_2.h5','r')


In [3]:
m=manager.ProductManager.from_config('/media/zahra/DATA/5by5_dishvar_pt1/prod_params_custom.yaml')
m_orig= manager.ProductManager.from_config('/media/zahra/DATA/example_5by5_lmax_200_no_scatter/prod_params_custom.yaml')
#m= manager.ProductManager.from_config('/media/zahra/DATA/example_5by5_lmax_200_no_scatter/prod_params_custom.yaml')


#correlation_arr,sum_counts,corr_counts=Bls_counts(m_orig)

#ant1_org=correlation_arr[:,0].astype(int)
#ant2_org=correlation_arr[:,1].astype(int)

#Scatterplot(m)
Ndish=25
t=m.telescope
x=t.feedpositions[:,0] #these are x and y positions not x and y polarizations
y=t.feedpositions[:,1]

t_orig = m_orig.telescope
x_orig = t_orig.feedpositions[:,0]
y_orig = t_orig.feedpositions[:,1]

Product directory: /media/zahra/DATA/5by5_dishvar_pt1/bt_matrices
Product directory: /media/zahra/DATA/example_5by5_lmax_200_no_scatter/bt_matrices


In [4]:
x

array([ 1.84012067e-01,  5.06937153e-03,  1.08378587e-01, -9.52308118e-02,
       -1.93400998e-02,  5.79113702e+00,  6.07852483e+00,  5.94309192e+00,
        5.95693161e+00,  5.98888204e+00,  1.21186920e+01,  1.19937621e+01,
        1.20256836e+01,  1.20428426e+01,  1.19708741e+01,  1.76401849e+01,
        1.81197237e+01,  1.79855998e+01,  1.80257770e+01,  1.79717523e+01,
        2.38692751e+01,  2.41037625e+01,  2.39931595e+01,  2.38889881e+01,
        2.40766471e+01,  1.84012067e-01,  5.06937153e-03,  1.08378587e-01,
       -9.52308118e-02, -1.93400998e-02,  5.79113702e+00,  6.07852483e+00,
        5.94309192e+00,  5.95693161e+00,  5.98888204e+00,  1.21186920e+01,
        1.19937621e+01,  1.20256836e+01,  1.20428426e+01,  1.19708741e+01,
        1.76401849e+01,  1.81197237e+01,  1.79855998e+01,  1.80257770e+01,
        1.79717523e+01,  2.38692751e+01,  2.41037625e+01,  2.39931595e+01,
        2.38889881e+01,  2.40766471e+01])

In [5]:
xx,yy=np.meshgrid(x[:Ndish],y[:Ndish])
antvec=np.arange(x[:Ndish].size)
ant1_uv,ant2_uv=np.meshgrid(antvec,antvec)
umat=xx*2-((xx)*2).transpose()
#print umat
vmat=yy*2-((yy)*2).transpose()
isok=np.where(ant2_uv>ant1_uv)
ant1_org_uv=ant1_uv[isok]
ant2_org_uv=ant2_uv[isok]

u_org=umat[isok]
v_org=vmat[isok]

In [6]:
time_channel=400
Ndish=25
Nbls=Ndish*(Ndish-1)/2
random_pt1=np.load('random_1e-2.npy')
run=1

v1=np.zeros(2*Nbls)
v1[0::2]=1
v2=np.zeros(2*Nbls)
v2[1::2]=1
vecs=1.e3*np.vstack([v1,v2])
src=np.zeros(2*Nbls)



In [ ]:
runs=100
def fit_gains(m,file_no_gain,gain_file, file_with_gain,file_gain_noise, time_channel,Tsys,ndays,random):
    
    gain=gain_file['gain'][0,:Ndish,time_channel]
    sim_gains=np.append(gain.real,gain.imag)
    sim_gains=np.zeros(Ndish*2)
    sim_gains[0::2]=gain.real
    sim_gains[1::2]=gain.imag
    sim_gains_amp=sim_gains[0::2]
    sim_gains_phase=sim_gains[1::2]

    gg=np.zeros((runs,Ndish*2))
    for ind_run in range(runs):
        print ind_run
        arr=Visibilities_grid(m,file_no_gain,gain_file,file_with_gain, file_gain_noise,time_channel,Tsys,ndays)
        vis_org=arr[0][:,time_channel]
        sigma=arr[-1]
        noise_org=sigma**2*np.ones(2*Nbls)
        print sigma
        vis,u,v,noise,ant1,ant2,edges,_,_=corrcal2.grid_data(vis_org,u_org,v_org,noise_org,ant1_org_uv,ant2_org_uv, tol=0.0001)
        print edges.shape
        big_noise=np.zeros(2*noise.size)
        big_noise[0::2]=noise
        big_noise[1::2]=noise

        big_vis=np.zeros(2*vis.size)
        big_vis[0::2]=np.real(vis)
        big_vis[1::2]=np.imag(vis)

        mat=corrcal2.sparse_2level(big_noise,vecs,src,2*edges) #init


        mult=random_pt1+1.
        gvec=np.array([])
        for i in range(len(sim_gains)):
            gvec=np.append(gvec,sim_gains[i]+random_pt1[i])
            #gvec=np.append(gvec,sim_gains[i]*mult[i])

        fac=1.;
        normfac=1.
        asdf=fmin_cg(corrcal2.get_chisq, gvec*fac, corrcal2.get_gradient,(big_vis,mat,ant1,ant2,fac,normfac))
        #asdf=fmin_cg(corrcal2.get_chisq_dense,gvec*fac,corrcal2.get_gradient_dense,(data,diag_mat,vecs_mat,ant1,ant2,fac,normfac))
        fit_gains_run=asdf/fac
        gg[ind_run,:]=fit_gains_run
    return gg, sim_gains_amp

    
    
gg_2, sim_gains_amp_2=fit_gains(m,ts_2,rg_2,ag_2, gn_2,time_channel,50,1,random_pt1)    


0
(0.012207031977595827, 'sigma_gn')
0.012207031977595827
(301,)
         Current function value: 0.005656
         Iterations: 0
         Function evaluations: 64
         Gradient evaluations: 52
1
(0.012207031977595827, 'sigma_gn')
0.012207031977595827
(301,)
         Current function value: 0.005655
         Iterations: 0
         Function evaluations: 76
         Gradient evaluations: 64
2
(0.012207031977595827, 'sigma_gn')
0.012207031977595827
(301,)
         Current function value: 0.005655
         Iterations: 0
         Function evaluations: 49
         Gradient evaluations: 37
3
(0.012207031977595827, 'sigma_gn')
0.012207031977595827
(301,)
         Current function value: 0.005656
         Iterations: 0
         Function evaluations: 76
         Gradient evaluations: 64
4
(0.012207031977595827, 'sigma_gn')
0.012207031977595827
(301,)
         Current function value: 0.005655
         Iterations: 0
         Function evaluations: 88
         Gradient evaluations: 76
5
(0.01220

In [ ]:

#gg_corrcal, sim_gains_corrcal = gg_corrcal_pt1fluc_pt1, sim_gains_corrcal_pt1fluc_pt1

def gains_mean_and_rel_err(gg,sim_gains_amp):
    gain_std=(np.std(gg,axis=0)/np.sqrt(runs)).flatten()
    gain_mean=np.mean(gg,axis=0).flatten() #shape is 98, we take every second one for amp so shape is 49
    rec_gains_mean_amp=gain_mean[0::2]
    rec_gains_std_amp=gain_std[0::2]
    rel_err_amp=np.abs(rec_gains_mean_amp-sim_gains_amp)/sim_gains_amp

    rel_err_amp_mean=np.mean(rel_err_amp)
    rel_err_amp_std=np.std(rel_err_amp,ddof=1)
    return rec_gains_mean_amp, rec_gains_std_amp, rel_err_amp, rel_err_amp_mean, rel_err_amp_std

rec_gains_mean_amp_2,_,rel_err_amp_2, rel_err_amp_mean_2 ,rel_err_amp_std_2 = gains_mean_and_rel_err(gg_2, sim_gains_amp_2)

print rel_err_amp_mean_2, rel_err_amp_std_2

In [ ]:
#np.save('gg_corrcal_dishvar_pt1m_nogainfluc_initguess_1e-4_tol_pt001',gg_2)
#np.save('simgains_corrcal_dishvar_pt1m_nogainfluc_initguess_1e-4_tol_pt001',sim_gains_amp_2)


In [ ]:
fig, ax = plt.subplots()

ax.scatter(sim_gains_amp_2, rec_gains_mean_amp_2)
#ax.plot(lims, lims,'g')
#plt.errorbar(x_true_real[:Ndish,time_channel],x_rec_real[:Ndish,time_channel],xerr=None,yerr=error[:Ndish],linestyle="None")
plt.xlabel('Simulated gains')
plt.ylabel('Recovered gains')
#plt.savefig('avgd_recgains_vs_simgains_corrcal_5by5_nogainfluc_Tsys_50_pt1_dish_scatter_1e-4_initguess_tol_pt001')
#lim=.002
#plt.ylim(0.9,1.025)
#plt.ylim(0.998,1.002)
print rec_gains_mean_amp_2

In [ ]:
#colour_scatterplot(m,rel_err_amp_2)
#plt.savefig('colourscatter_corrcal_dishscatter_1m_nogainfluc_initguess_pt1_tsys_50K.png')
plt.scatter(x[:Ndish],y[:Ndish],c=rel_err_amp_2,s=200) #, vmax=0.002)
plt.xlabel('Antenna x-location',fontsize=12); plt.ylabel('Antenna y-location',fontsize=12);plt.title('Relative amplitude error')
#plt.savefig('colour_scatplot_avgd_rel_err_corrcal_5by5_nogainfluc_Tsys_50_pt1_dish_scatter_1e-4_initguess_tol_pt001')
plt.colorbar()
plt.show()


In [ ]:
diff_x= x-x_orig
diff_y = y-y_orig
plt.scatter(rel_err_amp_2, diff_x[:Ndish])
plt.scatter(rel_err_amp_2, diff_y[:Ndish])
plt.ylabel('Difference in scatter')
plt.legend(('x', 'y'))
plt.xlabel('Relative amplitude error')
#plt.savefig('dish_scat_diff_vs_simgains_corrcal_5by5_nogainfluc_Tsys_50_pt1_dish_scatter_1e-4_initguess_tol_pt001')

In [ ]:
def hist_rel_err_mean_std(gg,sim_gains_amp):
    gg_amp=gg[:,0::2]
    rel_error=(np.abs(gg_amp-sim_gains_amp))/sim_gains_amp
    gg_amp_mean=np.mean(rel_error,axis=1) #shape is number of runs
    gg_amp_std=np.std(rel_error,axis=1, ddof=1)
    print gg_amp_mean
    print gg_amp_std
    return gg_amp_std, gg_amp_mean


In [ ]:
std, mean = hist_rel_err_mean_std(gg_2,sim_gains_amp_2)
plt.hist(std,'auto')
plt.xlabel('Standard deviation')
#plt.xscale('log')
#plt.xticks(np.linspace(6.16e-5,6.22e-5,5))
#plt.xticks(np.linspace(std.min(),std.max(),4))

#plt.title('100 runs, dish scat pt1m, no gain fluc, Tsys 50, init guess pt1, tol 0.05')
#plt.savefig('std_hist_corrcal_5by5_nogainfluc_Tsys_50_dish_scatter_1e-4_initguess_pt1_tol_pt001')
plt.show()

In [ ]:
plt.hist(mean,'auto')
plt.xlabel('Mean')
#plt.xscale('log')
#plt.xticks(np.linspace(mean.min(),mean.max(),4))

#plt.savefig('mean_hist_corrcal_5by5_nogainfluc_Tsys_50_dish_scatter_1e-4_initguess_pt1_tol_pt001')
plt.show()